In [1]:
import sys
sys.path.append("/home/zhouzr/project/Task-Oriented-Chatbot")
import pandas as pd
import numpy as np
from chatbot.models.word2vec import Word2vecExt
from chatbot.preprocessing.text import cut

import pysparnn.cluster_index as ci

from sklearn.feature_extraction.text import TfidfVectorizer

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.482 seconds.
Prefix dict has been built succesfully.


In [2]:
word2vec = Word2vecExt()
word2vec.load("/home/zhouzr/project/Task-Oriented-Chatbot/results/word2vec/word2vec.model")

2018-05-25 14:18:56,096 - INFO - Time Counter - load running 0.06 min


In [120]:
policy = pd.read_csv("/home/zhouzr/project/Task-Oriented-Chatbot/corpus/policy_file_etl.csv")
policy["content"] = policy.name + policy.department + policy.label

In [74]:
policy["content"] = policy.content.apply(cut).apply(lambda x: " ".join(x))

In [76]:
tfidf = TfidfVectorizer()
tfidf.fit(policy.content)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [84]:
cp = ci.MultiClusterIndex(content_tfidf, policy.name.tolist())

In [144]:
def query(q, n=1):
    q_tfidf = tfidf.transform([" ".join(cut(q))]).toarray()
    return cp.search(q_tfidf, k=n,return_distance=True,)

In [147]:
query("结算相关文档", n=3)

[[('0.814239922452', '《关于调整四川电网丰枯峰谷电价政策有关事项的通知》'),
  ('0.840928044053', '《2018年四川电力交易指导意见》')]]

In [142]:
query("四川直接交易，集中竞价的政策文件", n=3)

[[('0.437583493608', '《关于开展2018年常规直接交易年度集中竞价交易的公告》'),
  ('0.533048026501', '《关于申报2018年常规直接交易年度集中竞价交易电量需求的公告》'),
  ('0.585385775952', '《2018年年度电力直接交易1号公告》')]]

In [148]:
query("跨省跨区", n=3)

/home/zhouzr/anaconda3/lib/python3.6/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)


[[('1.0', '《关于2015年直购电试点工作有关事项的通知》'),
  ('1.0', '《关于2018年直接交易年度双边交易电量合规校核结果的公告》'),
  ('1.0', '《关于申报2018年常规直接交易年度集中竞价交易电量需求的公告》')]]

In [ ]:
跨省跨区 现货市场 月度竞价 交易公告 交易结束公告 结算 供需比例 国网范围内的相邻省份 全国 北京

In [149]:
policy.to_csv("/home/zhouzr/project/Task-Oriented-Chatbot/corpus/policy_file.utf8.csv", index=None, encoding="utf8")

In [150]:
policy.to_csv("/home/zhouzr/project/Task-Oriented-Chatbot/corpus/policy_file.gbk.csv", index=None, encoding="gbk")